In [1]:
pip install jmetalpy

     |████████████████████████████████| 110 kB 5.3 MB/s 
  Created wheel for jmetalpy: filename=jmetalpy-1.5.5-py3-none-any.whl size=196212 sha256=4024f4f0d4cf138c556be7c3e3aef75470d12a185e9f67a024f308e0667d7dbd
  Stored in directory: /root/.cache/pip/wheels/ca/8a/fd/c7b0b43039ee307e366569834484b5aa51ff81d609288dfcd2
Successfully built jmetalpy


Simulação

In [2]:
import random

VARIATION = 20

# order: Hp, Atk, Def, Agi, Esqv, Crit

def calcInitiative(agiPlayer, agiMonster):
    speedPlayer = agiPlayer + random.randrange(int(5+agiPlayer/4))
    speedMonster = agiMonster + random.randrange(int(5+agiMonster/4))
    return speedPlayer > speedMonster

def calcDano(atq_Atacante, def_Defensor, variation, taxaCritico):
    damage = 4*atq_Atacante - 2*def_Defensor
    newDamage = damage + ((random.uniform(0,0.4) - 0.2) * damage)
    if (random.randrange(0,100) < taxaCritico):
        return 3 * newDamage
    return newDamage
    
def calcEsquiva(taxaEvasao):
    return random.randrange(0,100) < taxaEvasao

# order: Hp, Atk, Def, Agi, Esqv, Crit
def Batalha(Player: list, Monster: list):
  player = Player.copy()
  monster = Monster.copy()
  player = [int(elem) for elem in player]
  monster =[int(elem) for elem in monster]
  lifePlayer = player[0]
  lifeMonster = monster[0]
  while True:
    if (calcInitiative(player[3], monster[3])):
      if (not calcEsquiva(monster[4])):
        #print(f'A vida do monstro era {lifeMonster}')
        danoPlayer = calcDano(player[1], monster[2], VARIATION ,player[5])
        lifeMonster -= danoPlayer
        #print(f'O Jogador começou e teve um dano de {danoPlayer}, o monstro ficou com {lifeMonster}')
        if lifeMonster <= 0:
          return True
      if (not calcEsquiva(player[4])):
        #print(f'A vida do jogador era {lifePlayer}')
        danoMonster = calcDano(monster[1], player[2], VARIATION, monster[5])
        lifePlayer -= danoMonster
        #print(f'O Jogador começou e o monstro deu dano de {danoMonster}, o player ficou com {lifePlayer}')
        if lifePlayer <= 0:
          return False
    else:
      if (not calcEsquiva(player[4])):
        #print(f'A vida do Player era {lifePlayer}')
        danoMonster = calcDano(monster[1], player[2], VARIATION, monster[5])
        lifePlayer -= danoMonster
        #print(f'O Monstro começou e teve um dano de {danoMonster}, o player ficou com {lifePlayer}')
        if lifePlayer <= 0:
          return False
      if (not calcEsquiva(monster[4])):
        #print(f'A vida do monster era {lifeMonster}')
        danoPlayer = calcDano(player[1], monster[2], VARIATION ,player[5])
        lifeMonster -= danoPlayer
        #print(f'O Monstro começou e o jogador deu dano de {danoPlayer}, o monstro ficou com {lifeMonster}')
        if lifeMonster <= 0:
          return True

def SimulateBattles(player: list, monster: list, numSims: int) -> float:
  playerWins = 0
  for a in range(0, numSims):
    if (Batalha(player, monster)):
      playerWins += 1
  return playerWins/numSims 


In [3]:
def constraints(monster: list, monsterProx: list, player: list, first: bool):
  if first:
    return [[int(monster[0]*0.5), int((monster[1]/2)) + 1, int(monster[2]*0.5), int(monster[3]*0.5), int(monster[4]*0.5), int(monster[5]*0.5) ], [int(monsterProx[0]), int(monsterProx[1]), int(2*monsterProx[1]) -1, int(monsterProx[3]), int(monsterProx[4]), int(monsterProx[5])]]
  else:
    return [[player[0], player[1], player[2], player[3], player[4], player[5] ], [int(monsterProx[0]), int(monsterProx[1]), int(2*monsterProx[1]) -1, int(monsterProx[3]), int(monsterProx[4]), int(monsterProx[5])] ]

#constraints([100,20,35], [987, 79, 57], [153, 24, 22], False )

Problema

In [4]:
from jmetal.core.quality_indicator import QualityIndicator
from jmetal.core.problem import FloatProblem
from jmetal.core.problem import FloatSolution
import random
class FitnessObjective(QualityIndicator):
    def __init__(self, is_minimization: bool = True):
        super(FitnessObjective, self).__init__(is_minimization=is_minimization)

    def compute(self, solution: FloatSolution) -> float:
        #print(solution.objectives[0] * (1 if self.is_minimization else -1))
        print(solution)
        return solution.objectives[0] * (1 if self.is_minimization else -1)

    def get_name(self) -> str:
        return 'Fitness'

    def get_short_name(self) -> str:
        return 'Fitness'

In [5]:


class Attributes(FloatProblem):

    # order: Hp, Atk, Def

    def __init__(self, desiredWinRatio: float, monster: list, lower_bound: list, upper_bound: list):
        super(Attributes, self).__init__()
        self.desiredWinRatio = desiredWinRatio
        self.monster = monster

        self.lower_bound = lower_bound
        self.upper_bound = upper_bound

        self.number_of_objectives = 1
        self.number_of_variables = 6
        self.number_of_constraints = 0

        self.first_solutions = 0

    def evaluate(self, solution: FloatSolution) -> FloatSolution:
        simWinRatio = SimulateBattles(solution.variables, self.monster, 10000)
        solution.objectives[0] = abs(simWinRatio - self.desiredWinRatio) if (simWinRatio != 0.0 and simWinRatio != 1.0) else 1.0
        return solution

    def create_solution(self) -> FloatSolution:
        
        new_solution = FloatSolution(
            self.lower_bound,
            self.upper_bound,
            self.number_of_objectives,
            self.number_of_constraints)
        
        if (self.first_solutions > 0):
            new_solution.variables = self.monster
            self.first_solutions -= 1
        else:
            new_solution.variables = \
                [int(random.uniform(self.lower_bound[i] * 1.0, self.upper_bound[i] * 1.0))
                for i in range(self.number_of_variables)]

        return new_solution

    def get_name(self) -> str:
        return 'Attributes'

Solve the problem

In [6]:
listMonsters = [[100,20,35,0,0,0], [987, 79, 57,5,1,2], [1996, 103, 100,10,2,4], [2800, 158, 170,12,4,6], [4155, 200, 200,15,6,9], [4700, 272, 262 ,23,7,10], [5153, 317, 320,31,10,13], [6239, 379, 430,45,13,16], [7356, 478, 503,50,15,20], [8980, 578, 600,64,18,27], [9999,999,999, 99, 99, 99]]

In [ ]:
from jmetal.algorithm.singleobjective.genetic_algorithm import GeneticAlgorithm
from jmetal.operator import BinaryTournamentSelection, PolynomialMutation, SBXCrossover
from jmetal.problem.singleobjective.unconstrained import Rastrigin
from jmetal.util.termination_criterion import StoppingByEvaluations, StoppingByQualityIndicator
from jmetal.core.operator import Mutation, Crossover, Selection


if __name__ == '__main__':
    time = 0
    first = True
    listSolutions = [[0,0,0,0,0,0]]
    for index in range(0,len(listMonsters)-1):
      constrain = constraints(listMonsters[index],listMonsters[index+1], listSolutions[index], first)
      print(constrain)
      problem = Attributes(0.8, monster=listMonsters[index], lower_bound=constrain[0], upper_bound= constrain[1])
      algorithm = GeneticAlgorithm(
          problem=problem,
          population_size=100,
          offspring_population_size=100,
          mutation=PolynomialMutation(1.0/problem.number_of_variables, 20.0),
          crossover=SBXCrossover(0.9, 20.0),
          selection=BinaryTournamentSelection(),
          #termination_criterion=StoppingByEvaluations(max_evaluations=30000)
          termination_criterion=StoppingByQualityIndicator(quality_indicator = FitnessObjective(is_minimization = True), expected_value = 0.05, degree = 1),
      )

      algorithm.run()
      result = algorithm.get_result()
      print('Solution: {}'.format(result.variables))
      listSolutions.append(result.variables)
      time += algorithm.total_computing_time
      first = False

[[50, 11, 17, 0, 0, 0], [987, 79, 157, 5, 1, 2]]


In [ ]:
print(time)

In [ ]:
print(listSolutions)

In [ ]:
playersNv = [[96.95175039829985, 31.908099267330165, 17.059027224501023, 0, 0, 0], [1338, 88.89445122785052, 18, 6.249553066996842, 1, 3], [1532, 92.2536064448443, 149, 10, 1, 5.036941306292804], [2075, 180, 199, 14, 3, 8], [3356.475724916584, 237.0118308942317, 232.47968430878973, 15, 5, 9], [3708, 262, 376, 27, 8, 9], [4843.035907026777, 283.7888729247261, 444, 39.076483381612135, 8.971355650573702, 12.009550992996797], [6768.4915278055305, 355, 513.8124408933611, 47.11817316891198, 14.01974183818226, 15.066327076320166], [6835.814217503189, 509.0071264893897, 625, 52.31844931780542, 14.071218451948251, 16], [7277.579085657899, 609, 628, 69, 38, 35.62439797650703]]
listMonsters = [[100,20,35,0,0,0], [987, 79, 57,5,1,2], [1996, 103, 100,10,2,4], [2800, 158, 170,12,4,6], [4155, 200, 200,15,6,9], [4700, 272, 262 ,23,7,10], [5153, 317, 320,31,10,13], [6239, 379, 430,45,13,16], [7356, 478, 503,50,15,20], [8980, 578, 600,64,18,27]]

playersNvInt = [[64, 41, 22, 1, 0, 0], [1722, 58, 64, 9, 0, 1], [2268, 118, 81, 10, 2, 3], [2673, 135, 238, 10, 4, 4], [4652, 185, 242, 22, 6, 6], [5015, 313, 244, 24, 9, 10], [5869, 340, 338, 31, 9, 12], [6059, 404, 510, 43, 11, 12], [8192, 457, 611, 49, 15, 24], [8770, 520, 722, 84, 27, 43]]

for a in range(0, len(playersNv)):
  print(SimulateBattles(playersNvInt[a], listMonsters[a], 10000))